In [1]:
import numpy as np
import cv2
import PIL
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras import ops
from tensorflow.keras.models import Sequential

In [2]:
!unzip data.zip

unzip:  cannot find or open data.zip, data.zip.zip or data.zip.ZIP.


In [ ]:
data_dir = "/content/data"

In [ ]:
import pathlib
data_dir = pathlib.Path(data_dir)
data_dir

PosixPath('/content/data')

In [ ]:
red_base = list(data_dir.glob("red_base/*.jpg"))
PIL.Image.open(str(red_base[10]))
img = cv2.imread(str(red_base[10]))
img.shape

(260, 260, 3)

In [ ]:
base_images ={
    "red_base" : list(data_dir.glob("red_base/*.jpg")),
    "blue_base" :list(data_dir.glob("blue_base/*.jpg")),
    "no_base" :list(data_dir.glob("no_base/*.jpg"))

}
base_labels ={
    "red_base" :0,
    "blue_base" : 1,
    "no_base" : 2

}

In [ ]:
x,y = [] , []
for base_name , images in base_images.items() :
  for image in images :
    img=cv2.imread(str(image))
    x.append(img)
    y.append(base_labels[base_name])

In [ ]:
x=np.array(x)
y= np.array(y)

[[[[141 153 131]
   [141 152 132]
   [142 152 136]
   ...
   [155 158 156]
   [157 160 158]
   [158 161 159]]

  [[142 154 132]
   [142 153 133]
   [143 153 137]
   ...
   [157 160 158]
   [159 162 160]
   [159 162 160]]

  [[143 154 134]
   [143 153 136]
   [144 154 138]
   ...
   [159 163 158]
   [159 163 158]
   [159 163 158]]

  ...

  [[166 170 164]
   [166 171 162]
   [165 170 161]
   ...
   [121 104 117]
   [ 99  82  96]
   [106  89 103]]

  [[166 170 164]
   [166 171 162]
   [165 170 161]
   ...
   [ 96  88  99]
   [ 84  75  88]
   [ 94  85  98]]

  [[166 170 164]
   [166 171 162]
   [165 170 161]
   ...
   [134 131 140]
   [102  98 110]
   [ 89  85  97]]]


 [[[158 166 149]
   [159 167 150]
   [157 164 151]
   ...
   [160 164 152]
   [160 164 152]
   [161 165 153]]

  [[158 166 149]
   [159 167 150]
   [158 165 152]
   ...
   [159 164 149]
   [160 164 152]
   [161 166 151]]

  [[157 165 148]
   [158 166 149]
   [158 165 152]
   ...
   [164 167 151]
   [165 168 153]
   [166 169

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, random_state =0)

In [ ]:
x_train= x_train/255
x_test= x_test /255

CNN without data augmentation

In [ ]:
cnn=tf.keras.models.Sequential([tf.keras.layers.Conv2D(16,3,padding= "same",activation= tf.nn.relu,input_shape=(260,260,3)),
                                  tf.keras.layers.MaxPooling2D(),
                                tf.keras.layers.Conv2D(32,3,padding= "same",activation= tf.nn.relu),
                                  tf.keras.layers.MaxPooling2D(),
                                tf.keras.layers.Conv2D(64,3,padding= "same",activation= tf.nn.relu),
                                  tf.keras.layers.MaxPooling2D(),
                                tf.keras.layers.Conv2D(128,3,padding= "same",activation= tf.nn.relu),
                                  tf.keras.layers.MaxPooling2D(),
                                layers.Dropout(0.2),

                                  tf.keras.layers.Flatten(),
                                 tf.keras.layers.Dense(256,activation= tf.nn.relu),
                                  tf.keras.layers.Dense(128,activation= tf.nn.relu),
                                  #tf.keras.layers.Dense(64,activation= tf.nn.relu),
                                  tf.keras.layers.Dense(16,activation= tf.nn.relu),
                                  tf.keras.layers.Dense(3,activation = tf.nn.softmax)])
cnn.compile(optimizer = "adam", loss = "sparse_categorical_crossentropy",metrics = ["accuracy"])
cnn.fit(x_train,y_train,epochs=20)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20


KeyboardInterrupt: 

In [ ]:
cnn.evaluate(x_test,y_test)

3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 412ms/step - accuracy: 0.6955 - loss: 0.4645


[0.4743783473968506, 0.6800000071525574]

In [ ]:
data_augmentation = Sequential(
                              [
                               layers.RandomRotation(0.1),
                               layers.RandomZoom(0.1)
                               ]
                               )

Different version with data augmentation

In [ ]:
cnn=tf.keras.models.Sequential([data_augmentation,
                                tf.keras.layers.Conv2D(16,3,padding= "same",activation= tf.nn.relu,input_shape=(260,260,3)),
                                  tf.keras.layers.MaxPooling2D(),
                                tf.keras.layers.Conv2D(32,3,padding= "same",activation= tf.nn.relu),
                                  tf.keras.layers.MaxPooling2D(),
                                tf.keras.layers.Conv2D(64,3,padding= "same",activation= tf.nn.relu),
                                  tf.keras.layers.MaxPooling2D(),
                                tf.keras.layers.Conv2D(128,3,padding= "same",activation= tf.nn.relu),
                                  tf.keras.layers.MaxPooling2D(),
                                #layers.Dropout(0.2),

                                  tf.keras.layers.Flatten(),
                                  tf.keras.layers.Dense(128,activation= tf.nn.relu),
                                  tf.keras.layers.Dense(64,activation= tf.nn.relu),
                                  tf.keras.layers.Dense(16,activation= tf.nn.relu),
                                  tf.keras.layers.Dense(3,activation = tf.nn.softmax)])
cnn.compile(optimizer = "adam", loss = "sparse_categorical_crossentropy",metrics = ["accuracy"])
cnn.fit(x_train,y_train,epochs=20)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 24s 2s/step - accuracy: 0.3542 - loss: 1.0969
Epoch 2/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - accuracy: 0.7014 - loss: 0.7143
Epoch 3/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.6299 - loss: 0.5016
Epoch 4/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.6543 - loss: 0.4543
Epoch 5/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.6899 - loss: 0.4818
Epoch 6/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - accuracy: 0.6547 - loss: 0.5233
Epoch 7/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.6688 - loss: 0.5020
Epoch 8/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.6691 - loss: 0.4905
Epoch 9/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.6893 - loss: 0.5059
Epoch 10/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - accuracy: 0.6761 - loss: 0.4677
Epoch 11/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - accuracy: 0.6753 - loss: 0.4802
Epoch 12/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - accuracy: 0.6297 - loss: 0.4674
Epoch 13/20
8

In [ ]:
cnn.summary()

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_6 (Sequential)       │ (None, 260, 260, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_46 (Conv2D)              │ (None, 260, 260, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_46 (MaxPooling2D) │ (None, 130, 130, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_47 (Conv2D)              │ (None, 130, 130, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_47 (MaxPooling2D) │ (None, 65, 65, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_48 (Conv2D)              │ (None, 65, 65, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_48 (MaxPooling2D) │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_49 (Conv2D)              │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_49 (MaxPooling2D) │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_12 (Flatten)            │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_41 (Dense)                │ (None, 128)            │     4,194,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_42 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_43 (Dense)                │ (None, 16)             │         1,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_44 (Dense)                │ (None, 3)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,903,659 (49.22 MB)

 Trainable params: 4,301,219 (16.41 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 8,602,440 (32.82 MB)

In [ ]:
model_save_path = 'puck_collector.keras'
cnn.save(model_save_path)
print(f"\nModel saved successfully to: {model_save_path}")


Model saved successfully to: puck_collector.keras
